# 랭체인 입문

LLM powered application 제작을 위한 프레임워크

## 랭체인 기초

In [3]:
# %pip install -U langchain langchain-openai

In [5]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')


In [44]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-5-nano')
llm.invoke("Hello, world!")

AIMessage(content='Hello! Nice to meet you. If you’re just testing, I’m here to help with whatever you need. Want a quick Hello, World! example in a specific language, or something else? Here are a few quick snippets:\n\n- Python: print("Hello, World!")\n- JavaScript: console.log("Hello, World!")\n- C: #include <stdio.h>\n  int main(void) { printf("Hello, World!"); return 0; }\n\nTell me what you’re trying to do, and I’ll tailor it.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 822, 'prompt_tokens': 10, 'total_tokens': 832, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 704, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CArY3UtrKsHxsXecQVJrdKz6acDJe', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--487dc7fd-a43e-4c5e

In [ ]:
# 외국어 -> 한국어로 번역하는 작업

msg = input('외국어')

res = llm.invoke(f'다음 내용을 한국어로 번역해줘: {msg}')

res.content

'What should I eat for lunch?'

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    # 채팅 세션의 전체적인 안내사항
    SystemMessage(content='다음 내용을 영어로 번역해줘'),
    HumanMessage(content='점심 뭐먹지')
]

res = llm.invoke(messages)
res.content

'What should I eat for lunch?'

In [ ]:
messages = [
    # SystemMessage(content='다음 내용을 영어로 번역해줘'),
    {'role': 'system', 'content': '다음 내용을 영어로 번역해줘'},
    # HumanMessage(content='점심 뭐먹지'),
    {'role': 'user', 'content': '점심 뭐먹지'}
]

res = llm.invoke(messages)
res.content

'What should I have for lunch?'

In [ ]:
for token in llm.stream(messages):
    print(token.content, end='|')
    
# 일반적으로 답변 생성 시 토큰별로 생성해서 보여주는 편

|What| should| I| have| for| lunch|?||

## 프롬프트 템플릿

고정된 문자열과 변수를 조합해 프롬프트를 만듦

In [29]:
from langchain_core.prompts import ChatPromptTemplate

messages = [
    {'role': 'system', 'content': 'translate korean to {lang}'},
    {'role': 'user', 'content': '{text}'}
]

prompt_template = ChatPromptTemplate.from_messages(messages)
prompt = prompt_template.invoke({'lang': 'japanese', 'text': '오늘 점심 맛있었어'})
# 이 전체가 묶여서 템플릿으로 만들어진것

prompt.to_messages()
llm.invoke(prompt)

AIMessage(content='今日の昼ご飯、美味しかったですね。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 30, 'total_tokens': 49, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CAqctpUGCR22UBYOXLCZ8aB0hCr1P', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--5cf50841-71de-4577-8ff7-a8d196d809f0-0', usage_metadata={'input_tokens': 30, 'output_tokens': 19, 'total_tokens': 49, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
# 템플릿이 있으니까 일부만 바꿔도 알아서 전체에 적용해준다.
prompt = prompt_template.invoke({'lang': 'Deutsch', 'text': '프랑스나 독일이나 거기서 거기임'})
llm.invoke(prompt)

AIMessage(content='Das ist egal, ob es Frankreich oder Deutschland ist.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 35, 'total_tokens': 47, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CAqgqgf9xT1kxY1AQjoTowYkXDMBO', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--7d8fb9ee-4938-4651-98ca-8078be78340e-0', usage_metadata={'input_tokens': 35, 'output_tokens': 12, 'total_tokens': 47, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## 체인
- langchain의 각 구성요소를 묶어서(Chaining) 한번에 실행(invoke) 할 수 있는 기능
- `a | b | c | ...`의 형태 -> 파이썬 문법이 아니라, 랭체인 문법(LangChain Expression Language, LCEL)
- 기본적으로 프롬프트부터 시작함
- 파이프 안에 다른게 계속 들어가짐

In [ ]:
# 체인 -> 원하는 조건을 프롬프트 템플릿에 적용 - 템플릿으로 AI에서 결과를 만듦 - 결과물 해석까지 연이어서 할 수 있다

from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
chain = prompt_template | llm | output_parser
chain.invoke({'lang': 'Welsh ', 'text': '웨일스에 독립을'})

'annibyniaeth i Gymru'

In [47]:
# ChatPromptTemplate -> 채팅을 할 때 씀
# PromptTemplate -> 한번, 단발성으로 사용할 때 쓴다

from langchain_core.prompts import PromptTemplate

template = '''
당신은 클라이언트의 요구 사항을 분석해서 단계를 나눠주는 데이터 분석 전문가입니다.
사용자의 질문을 EDA에 활용할 수 있도록 단계를 나누어 주세요.

질문: {question}
'''

prompt = PromptTemplate.from_template(template)

chain = prompt | llm | output_parser

res = chain.invoke({'question': '25년 전반기 매출 분석'})

print(res)

다음은 “25년 전반기 매출 분석”을 EDA(탐색적 데이터 분석) 관점에서 단계별로 나눈 실행 로드맷입니다. 각 단계는 실제 분석 흐름에 맞춰 구체적 작업으로 구성했습니다.

1) 분석 목표 정의 및 범위 설정
- 기간 정의: 2025년 1월 1일 ~ 2025년 6월 30일(H1 2025)
- 핵심 지표(KPI)
  - 매출액(Revenue), 주문건수(Orders), 평균 주문가(AOV)
  - 카테고리별 매출, 지역별 매출
  - 원가/이익(Gross Profit, Margin)
  - 채널/프로모션 영향 분석 여부
- 비교 기준
  - 2024년 H1 또는 최근 비슷한 기간과의 YoY 비교
- 산출물 형식
  - 요약 인사이트 리포트, 데이터 품질 로그, 재현 가능한 노트북/스크립트, 시각화 대시보드 초안

2) 데이터 소스 파악 및 품질 진단
- 필요한 데이터 아이템(예시)
  - 거래/매출 데이터: OrderDate, Revenue, Quantity, UnitPrice, Discount, Cost, Profit, OrderID
  - 차원 데이터: CustomerRegion, ProductCategory, Channel, PromotionFlag, ProductID/Name
  - 마스터 데이터(선택): Customer, Product, Region/Stores
- 데이터 품질 점검
  - 누락(missing), 중복(duplicate), 기간 필터링(2025년 H1에 해당하는 주문만)
  - 통화/통화 단위의 일관성, 음수값 여부, 이상치(outliers) 여부
  - 날짜 형식 및 시계열 정합성 확인
- 데이터 흐름 문서화
  - 소스-변환-로딩(ETL) 파이프라인 개략도 작성

3) 데이터 준비 및 구조화
- 데이터 통합
  - 거래 데이터와 차원 데이터(카테고리, 지역, 채널, 프로모션) 조인
- 파생 변수 생성
  - Year, Month, Day, HalfYear(H1/H2) 또는 PeriodLabel(2025-01, 2025-02, …)


In [45]:
# Stream
for token in chain.stream({'question': '보편적인 분석 프로젝트 진행'}):
    print(token, end='^', flush=True)
    
# 일반적인 LLM에서 답변을 보여주는 방식이 이렇다

^1^.^ 요^구^ 사^항^ 분^석^
^  ^ -^ 클^라^이^언^트^가^ 원^하는^ 분^석^ 목^표^ 및^ 목^적^ 파^악^
^  ^ -^ 데이터^ 수^집^ 및^ 가^용^성^ 확인^
^  ^ -^ 클^라^이^언^트^가^ 원^하는^ 결과^물^ 및^ 보^고^서^ 형^식^ 결^정^
   
^2^.^ 데이터^ 전^처^리^
^  ^ -^ 데이터^ 수^집^ 및^ 크^기^ 확인^
^  ^ -^ 데이터^의^ 결^측^치^,^ 이^상^치^,^ 중^복^값^ 등^의^ 처리^
^  ^ -^ 데이터^의^ 형^식^ 변^환^ 및^ 범^주^형^ 데이터^ 인^코^딩^
^  ^ -^ 데이터^의^ 스^케^일^링^ 및^ 정^규^화^
   
^3^.^ 탐^색^적^ 데이터^ 분^석^ (^EDA^)
^  ^ -^ 데이터^의^ 분^포^ 및^ 통^계^량^ 확인^
^  ^ -^ 변수^ 간^ 상^관^ 관^계^ 및^ 패^턴^ 파^악^
^  ^ -^ 데이터^ 시^각^화^를^ 통^한^ 인^사^이^트^ 도^출^
^  ^ -^ 특^징^ 선택^ 및^ 차^원^ 축^소^ 기^법^ 적^용^
   
^4^.^ 모^델^링^
^  ^ -^ 분^석^ 목^표^에^ 맞^는^ 모^델^ 선택^
^  ^ -^ 학^습^ 및^ 테^스트^ 데이터^ 셋^ 분^리^
^  ^ -^ 모^델^ 학^습^ 및^ 튜^닝^
^  ^ -^ 모^델^ 성^능^ 평^가^ 및^ 검^증^
   
^5^.^ 결과^ 해^석^ 및^ 보^고^
^  ^ -^ 모^델^ 결과^ 해^석^ 및^ 클^라^이^언^트^에^게^ 설^명^
^  ^ -^ 모^델^의^ 성^능^ 및^ 결과^물^ 평^가^
^  ^ -^ 결^론^ 도^출^ 및^ 추^후^ 전^략^ 제^안^
^  ^ -^ 보^고^서^ 작^성^ 및^ 발^표^

^이^와^ 같^은^ 단^계^를^ 따^라^가^면^ 보^다^ 체^계^적^이^고^ 효^율^적^인^ 데이터^ 분^석^ 프^로^젝^트^가^ 가능^할^ 것^입니다^.^^

In [ ]:
# batch는 몇개를 동시에 한 프롬프트 템플릿에 넣을 때 사용함

prompt = PromptTemplate.from_template('{topic}에 대해 3문장으로 설명해줘')
llm = ChatOpenAI(model='gpt-4.1-nano')

chain = prompt | llm | StrOutputParser()

chain.batch([
    {'topic': 'Langchain'},
    {'topic': 'Langsmith'},
    {'topic': 'Langgraph'}
])


['Langchain은 여러 가지 언어모델과 도구들을 연결하여 자연어 처리 애플리케이션을 쉽게 구축할 수 있도록 돕는 프레임워크입니다. 이를 통해 챗봇이나 자동화된 질문응답 시스템 등 복잡한 AI 기반 작업을 효율적으로 개발할 수 있습니다. 또한, 데이터 소스와의 통합과 사용자 맞춤형 응답 생성 등을 지원하여 다양한 산업 분야에서 활용되고 있습니다.',
 'Langsmith는 OpenAI가 개발한 강력한 언어 모델 플랫폼으로, 사용자들이 자연어 처리 작업을 쉽게 수행할 수 있도록 지원합니다. 이 플랫폼은 텍스트 생성, 요약, 번역 등의 다양한 언어 관련 기능을 통합하여 제공하며, 개발자들이 맞춤형 AI 애플리케이션을 구축할 수 있도록 돕습니다. 또한, 사용자 친화적인 인터페이스와 API를 통해 빠른 개발과 효율적인 통합이 가능하게 설계되어 있습니다.',
 'LangGraph는 자연어 처리(NLP) 분야에서 언어 구조를 그래프로 표현하는 기법입니다. 이를 통해 문장 내 단어와 의미 간의 관계를 시각화하고 분석할 수 있습니다. LangGraph는 언어 이해와 텍스트 분석의 정밀도를 높이기 위해 다양한 관계망과 연결 구조를 활용합니다.']